<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson S1 - Defining the model domain and initial conditions using the WRF Preprocessing System (WPS)

## Overview
This lesson covers creating your case-specific domain and defining static and initial conditions used for the land surface model within WRF-Hydro. 

### Software and input data
The uses the following software:
* WPS v4.2 built with the GNU compilers
* R environment including all libraries for running create_wrfinput and create_soilproperties scripts
* WPS geographical input data for the contiguous United States 

**Note:** Only USGS and default datasets are supported in this container. `Lai_modis_30s`, `nlcd2011_30m`, and `topo_30s` are subsets for training purposes only

## Defining the model domain and creating the geogrid file
WRF-Hydro leverages the extensive domain generation and geophysical dataset processing tools available in the [WRF Preprocessing System (WPS)](http://www2.mmm.ucar.edu/wrf/users/) to define model domains and generate input files for the land surface model. In particular WRF-Hydro requires the geogrid and file derived from these WPS utilities and a wrfinput file from WRF or a utility script provided by the WRF-Hydro team. 

This tutorial walks users through the process of manually creating a geogrid file from an existing WPS installation and creating wrfinput files from provided script packages.  

For those interested in the details of running the complete WPS system and generating wrfinput files through WRF, this is covered in our Docker training on the coupled WRF and WRF-Hydro modeling system.

### Data sources
The WPS geographical input data (e.g. land cover, soil texture, and terrain height) are some of the primary datasets used by WRF-Hydro. The entire dataset collection can be obtained from the WPS geographical input data [download page](http://www2.mmm.ucar.edu/wrf/users/download/get_sources_wps_geog.html). However, these data are rather large (~50GB) and contain a number of datasets that are not used in most implementations of WRF-Hydro. 

To facilitate the domain generation process, the WRF-Hydro team has created a subset of the data by removing unnecessary data, removing resolutions of the data not commonly used, and limiting the areal extent of this subset. This is consistent with the availability of other datasets used in this tutorial. Thus, the dataset provided with this container is considerably smaller than the complete WPS geographical input dataset.

### WPS geogrid
The WPS program `geogrid.exe` is used to create the `geo_em.d01.nc` file, hereafter referred to as the *geogrid* file. The `geogrid.exe` program takes a Fortran namelist (`namelist.wps`) defining the projection, resolution, and extent of the model domain as well as the desired input fields and the [WPS geographical input data](http://www2.mmm.ucar.edu/wrf/users/download/get_sources_wps_geog.html) as inputs and creates the geogrid file. 

### Defining domain boundaries
The land surface model in WRF-Hydro operates on the domain defined within the geogrid file. The `geogrid.exe` program generates this file based upon input variables specified in the `namelist.wps` file. 

The first step to creating the geogrid file is to define our domain boundaries. The `geogrid.exe` program takes by default a centerpoint x and y grid specification, and other projection information to define a bounding box for the domain. If you do not already have coordinates, you can obtain them from Google Earth or a standard GIS system. However, for this tutorial we have supplied the coordinates for you and you may use the WRF-Hydro geogrid Python command line utility to visualize the domain location and create the geogrid file given these coordinates.

### Exploring the WPS geogrid utility
The WPS `geogrid.exe` utility is controlled by options set in the `namelist.wps`. As previously stated, there are many options in the `namelist.wps` file that are not relevant to most WRF-Hydro users. For educational purposes, we will take a quick look at a full `namelist.wps` file and then discuss the relevant options used by the WRF-Hydro geogrid utility in more detail.

#### Setting up the lesson directory

**Step 1: Setup the directory for this lesson**

First, we need to create a directory to hold the inputs and outputs for this lesson.

In [ ]:
%%bash

mkdir -p ~/wrf-hydro-training/lesson-wps/geogrid

**Step 2: View the namelist** 
    
In the example case is a sample `namelist.wps` file for the domain of interest

In [ ]:
%%bash

cd ~/wrf-hydro-training/lesson-wps
cp ~/wrf-hydro-training/example_case/supplemental/namelist.wps .
cat namelist.wps

A full description of these options can be found in the [WPS User Guide](https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/v4.2/users_guide_chap3.html). 

The options that are of interest to most WRF-Hydro users are the following:

**Table 1. WPS namelist options relevent to WRF-Hydro**

|Option|Description|
|------|-----------|
| e_we | The number of grid edges in the west-east dimension. The number of grid centers will be e_we-1. |
| e_sn | The number of grid edges in the south-north dimension. The number of grid centers will be e_sn-1. |
| ref_lat | A real value specifying the latitude part of a (latitude, longitude) center-point of the domain. |
| ref_lon | A real value specifying the longitude part of a (latitude, longitude) center-point of the domain. West longitudes are negative, and the value of ref_lon should be in the range [-180, 180]. |
| dx | A real value specifying the grid distance in the x-direction where the map scale factor is 1. The grid distance is in meters for the 'polar', 'lambert', and 'mercator' projection, and in degrees longitude for the 'lat-lon' projection. |
| dy | A real value specifying the grid distance in the y-direction where the map scale factor is 1. The grid distance is in meters for the 'polar', 'lambert', and 'mercator' projection, and in degrees latitude for the 'lat-lon' projection. |
| map_proj | A character string specifying the projection of the simulation domain. Accepted projections are 'lambert', 'polar', 'mercator', and 'lat-lon'. Default value is 'lambert'. |
| truelat1 | A real value specifying, the first true latitude for the Lambert conformal conic projection, or the only true latitude for the Mercator and polar stereographic projections. |
| truelat2 | A real value specifying, the second true latitude for the Lambert conformal conic projection. For all other projections, truelat2 is ignored. No default value. |
| stand_lon | A real value specifying, the longitude that is parallel with the y-axis in the Lambert conformal and polar stereographic projections. For the regular latitude-longitude projection, this value gives the rotation about the earth's geographic poles. No default value. |
| geog_data_res | A character string specifying a corresponding resolution or list of resolutions separated by + symbols of source data to be used when interpolating static terrestrial data to the grid. This string should contain a resolution matching a string preceding a colon in a rel_path or abs_path specification (see the description of GEOGRID.TBL options) in the GEOGRID.TBL file for each field. If a resolution in the string does not match any such string in a rel_path or abs_path specification for a field in GEOGRID.TBL, a default resolution of data for that field, if one is specified, will be used. If multiple resolutions match, the first resolution to match a string in a rel_path or abs_path specification in the GEOGRID.TBL file will be used. Default value is 'default'. |
| geog_data_path | A character string giving the path, either relative or absolute, to the directory where the geographical data directories may be found. This path is the one to which rel_path specifications in the GEOGRID.TBL file are given in relation to. No default value. |

### Creating the geogrid file
In this section, we will create the geogrid file using the WPS geogrid utility. 

In [ ]:
%%bash

cd ~/wrf-hydro-training/lesson-wps
cp ~/WRF_WPS/WPS/geogrid/*.exe geogrid
cp ~/WRF_WPS/geog_conus/*TBL* geogrid/GEOGRID.TBL

./geogrid/geogrid.exe

### Inspecting the geogrid file
Now that we have created our geogrid file, we will quickly display the file header using the *ncdump* command line utility and open the file using the Python library `xarray` to visualize its contents. 

In [ ]:
%%bash

cd ~/wrf-hydro-training/lesson-wps
ncdump -h geo_em.d01.nc

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
xr.set_options(display_style="html")

ds = xr.open_dataset('~/wrf-hydro-training/lesson-wps/geo_em.d01.nc')
ds

In [ ]:
p = ds.HGT_M.plot()
p.axes.set_aspect('equal')

### Creating the wrfinput file

In [ ]:
%%bash

cd ~/wrf-hydro-training/lesson-wps
cp ~/wrf-hydro-training/wrf_hydro_model_tools/parameters/create_wrfinput/*.R .
Rscript create_wrfinput.R --geogrid="geo_em.d01.nc"

**View the *wrfinput_d01.nc* file**

In [ ]:
%%bash

cd ~/wrf-hydro-training/lesson-wps
ncdump -h wrfinput_d01.nc

### Inspecting the wrfinput file

In [ ]:
ds = xr.open_dataset('~/wrf-hydro-training/lesson-wps/wrfinput_d01.nc')
ds

In [ ]:
p = ds.SMOIS.sel(soil_layers_stag=0).plot()
p.axes.set_aspect('equal')

### Creating the soil properties file

In [ ]:
%%bash

cd ~/wrf-hydro-training/lesson-wps

cp ~/wrf-hydro-training/wrf_hydro_model_tools/parameters/create_soilproperties/*.R .

cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Land_models/NoahMP/run/*TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/HYDRO/*TBL .

Rscript create_soilproperties.R

### Inspecting the soil properties file

In [ ]:
ds = xr.open_dataset('~/wrf-hydro-training/lesson-wps/soil_properties.nc')
ds

In [ ]:
p = ds.smcmax.sel(soil_layers_stag=0).plot()
p.axes.set_aspect('equal')

This concludes the lesson on domain definition and initial conditions for WRF-Hydro. 

© UCAR 2020